# Import

In [1]:
import numpy as np
import networkx as nx
import pandas as pd
import itertools
import plotly.express as px
import plotly.graph_objects as go

# Setup

In [4]:
def graph_to_plotly(G):
    for node in G.nodes:
        T = len(G.nodes[node]['pos'][0])
    
    edges_pos ={'x': [], 'y': []}
    for t in range(T):
        edges_pos['x'].append([])
        edges_pos['y'].append([])
        for edge in G.edges:
            edges_pos['x'][t].append(G.nodes[edge[0]]['pos'][0][t])
            edges_pos['x'][t].append(G.nodes[edge[1]]['pos'][0][t])
            edges_pos['x'][t].append(None)
            
            edges_pos['y'][t].append(G.nodes[edge[0]]['pos'][1][t])
            edges_pos['y'][t].append(G.nodes[edge[1]]['pos'][1][t])
            edges_pos['y'][t].append(None)
    
    node_pos = {'x': [], 'y': []}
    for t in range(T):
        node_pos['x'].append([])
        node_pos['y'].append([])
        for node in G.nodes:
            node_pos['x'][t].append(G.nodes[node]['pos'][0][t])
            node_pos['y'][t].append(G.nodes[node]['pos'][1][t])
    
    return node_pos, edges_pos

In [5]:
def animate_network(node_pos, edge_pos):
    T = len(node_pos['x'])
    fig = go.Figure(
        data=[go.Scatter(x=node_pos['x'][0], y=node_pos['y'][0],
                        name="frame",
                        mode="markers",
                        line=dict(width=2, color="blue")),
            go.Scatter(x=edge_pos['x'][0], y=edge_pos['y'][0],
                        name="curve",
                        mode="lines",
                        line=dict(width=2, color="blue"))
            ],
        layout=go.Layout(width=600, height=600,
                        xaxis=dict(range=[-5, 5], autorange=False, zeroline=False),
                        yaxis=dict(range=[-5, 5], autorange=False, zeroline=False),
                        title="Moving Frenet Frame Along a Planar Curve",
                        hovermode="closest",
                        updatemenus=[dict(type="buttons",
                                        buttons=[dict(label="Play",
                                                        method="animate",
                                                        args=[None])])]
                        ),
        frames=[go.Frame(
            data=[go.Scatter(
                x= edge_pos['x'][k],
                y= edge_pos['y'][k],
                mode="lines",
                line=dict(color="red", width=2))
            ,
            go.Scatter(
                x= node_pos['x'][k],
                y= node_pos['y'][k],
                mode="markers",
                line=dict(color="blue", width=2))
            ]) for k in range(T)]
    )

    fig.show()

In [80]:
def log_force(p1, p2):
    eps = 0.00001
    r = np.linalg.norm(p1-p2)
    return -np.log(r+eps)/r*(p1-p2)

def log_square_force(p1, p2):
    eps = 0.00001
    r = np.linalg.norm(p1-p2)
    return -(np.log(r+eps+np.exp(r)-1)-1)/r*(p1-p2)

def gravity_force(p1, p2):
    eps = 0.00001
    r = np.linalg.norm(p1-p2)
    return -1/(r+eps)**3*(p1-p2)

def zero_force(p1, p2):
    return 0.0*(p1-p2)

def spring_force(p1, p2):
    eps = 0.00001
    r = np.linalg.norm(p1-p2)
    return (1-r)*(p1-p2)



In [81]:
def get_index_maps(G):
    node_to_index = {}
    index_to_node = {}
    current_i = 0
    for node in G.nodes:
        node_to_index[str(node)] = current_i
        index_to_node[current_i] = str(node)
        current_i += 1
    return node_to_index, index_to_node

In [82]:
def get_connection_matrix(G, node_to_index, num_nodes):
    connected = np.zeros((num_nodes, num_nodes), dtype=bool)
    for edge in G.edges:
        connected[node_to_index[edge[0]], node_to_index[edge[1]]] = True
        connected[node_to_index[edge[1]], node_to_index[edge[0]]] = True
    return connected

In [123]:
def simulate(G, num_frames=100, force_scale = 0.1, general_force=spring_force, connected_force=spring_force):
    num_nodes = G.number_of_nodes()
    node_to_index, index_to_node = get_index_maps(G)
    connected = get_connection_matrix(G, node_to_index, num_nodes)
    # init
    pos = np.zeros((2,G.number_of_nodes(), num_frames), dtype=float)
    vel = np.zeros((2,G.number_of_nodes()), dtype=float)
    acc = np.zeros((2,), dtype=float)
    pos[:, :, 0] = np.random.normal(0, np.sqrt(num_nodes), (2, num_nodes))
    (D, N, T) = pos.shape

    for t in range(T-1): 
        for n in range(N):
            acc[:] = 0
            for n2 in range(N):
                if n2 == n: continue
                acc[:] += force_scale*general_force(pos[:, n, t], pos[:, n2, t])
                if connected[n, n2]:
                    acc[:] += force_scale*connected_force(pos[:, n, t], pos[:, n2, t])
            

            vel[:, n] = (vel[:, n] + acc[:])
            pos[:, n, t+1] = pos[:, n, t] + vel[:, n]
            
    for node in G.nodes:
        G.nodes[node]['pos'] = pos[:, node_to_index[node], :]
                

# Ploting

In [124]:
G = nx.Graph()
N = 10
i = [i for i in range(N)]
s = [np.sin(i/6) for i in range(N)]
c = [np.cos(i/6) for i in range(N)]
t = [(i-N/2)/2 for i in range(N)]

G.add_node('A')
G.add_node('B')
G.add_node('C')
G.add_edge('A', 'B')
G.add_edge('B', 'A')
G.add_edge('A', 'C')
G.add_edge('C', 'A')


In [125]:
simulate(G, force_scale=0.01, general_force=log_force, connected_force=log_square_force)

In [126]:
node_pos, edge_pos = graph_to_plotly(G)
animate_network(node_pos, edge_pos)